In [1]:
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from tqdm import tqdm
from keras_tqdm import TQDMNotebookCallback

Using TensorFlow backend.


In [2]:
coin = pd.DataFrame.from_csv('data/history/BNBBTC.csv')
coin.head()

/Users/tkind/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning:

from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls



,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number Trades,Taker Base Asset Volume,Take Quote Asset Volume,Ignore
Open Time,,,,,,,,,,,
1516402800000,0.001305,0.001307,0.001302,0.001306,4945.79,1516403099999,6.452520,246,4093.20,5.340403,0
1516403100000,0.001307,0.001309,0.001303,0.001307,2427.36,1516403399999,3.173219,190,1087.44,1.422105,0
1516403400000,0.001307,0.001311,0.001303,0.001310,5777.02,1516403699999,7.557042,288,4332.20,5.668491,0
1516403700000,0.001311,0.001311,0.001308,0.001308,6382.58,1516403999999,8.349177,212,2819.08,3.687801,0
1516404000000,0.001308,0.001312,0.001308,0.001311,7349.22,1516404299999,9.617556,224,4900.57,6.412699,0


In [3]:
def plplot(d):
    data = []
    for l in d:
        data += [
            go.Scatter(
                x=list(range(0, len(l))), # assign x as the dataframe column 'x'
                y=l
            )
        ]
    py.iplot({'data': data})
plplot([coin['Open']])

In [4]:
x = []
for i in range(50, len(coin['Open'].values)):
    x += [coin['Open'].values[i-50:i]]
# x = x.reshape(1, -1)
x[50]

array([0.0013   , 0.0012962, 0.0013017, 0.0012998, 0.0012993, 0.0012999,
       0.001297 , 0.0012965, 0.0012985, 0.0012949, 0.0012971, 0.0012973,
       0.0012976, 0.0012976, 0.0013003, 0.0012949, 0.0012947, 0.0012941,
       0.001293 , 0.0012939, 0.001288 , 0.0012795, 0.0012644, 0.0012576,
       0.0012555, 0.0012582, 0.001285 , 0.0012697, 0.001268 , 0.001266 ,
       0.0012599, 0.0012605, 0.001257 , 0.0012654, 0.0012685, 0.0012755,
       0.0012816, 0.0012752, 0.0012783, 0.001278 , 0.0012778, 0.0012766,
       0.0012751, 0.001275 , 0.0012776, 0.0012765, 0.0012774, 0.0012754,
       0.0012755, 0.0012792])

In [5]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# this is the size of our encoded representations
encoding_dim = 5  # 32 floats
indim = len(x[0])

# this is our input placeholder
input_img = Input(shape=(indim,), name="in")
# "encoded" is the encoded representation of the input
encoded = Dense(int(indim/2), activation='linear')(input_img)
encoded = Dense(int(indim/2), activation='linear')(encoded)
encoded = Dense(int(indim/2), activation='linear')(encoded)
encoded = Dense(int(indim/3), activation='linear')(encoded)
encoded = Dense(int(indim/3), activation='linear')(encoded)
encoded = Dense(encoding_dim, activation='linear', name="ed")(encoded)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(int(indim/3), activation='linear')(encoded)
decoded = Dense(int(indim/3), activation='linear')(decoded)
decoded = Dense(int(indim/2), activation='linear')(decoded)
decoded = Dense(int(indim/2), activation='linear')(decoded)
decoded = Dense(int(indim/2), activation='linear')(decoded)
decoded = Dense(indim, activation='linear', name="out")(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (InputLayer)              (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_3 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                416       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
ed (Dense)                   (None, 5)                 85        
__________

In [6]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [7]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = Dense(int(indim/3), activation='linear')(encoded_input)
decoder_layer = Dense(int(indim/3), activation='linear')(decoder_layer)
decoder_layer = Dense(int(indim/2), activation='linear')(decoder_layer)
decoder_layer = Dense(int(indim/2), activation='linear')(decoder_layer)
decoder_layer = Dense(int(indim/2), activation='linear')(decoder_layer)
decoder_layer = Dense(indim, activation='linear', name="out")(decoder_layer)
# create the decoder model
decoder = Model(encoded_input, decoder_layer)

In [8]:
autoencoder.compile(optimizer='Adam', loss='mse')

In [16]:
his = autoencoder.fit([x], [x], epochs=1000, batch_size=16, validation_split=0.2, verbose=1,callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1)])

Train on 33439 samples, validate on 8360 samples
Epoch 1/1000
33439/33439 [==============================] - 7s 206us/step - loss: 4.9137e-09 - val_loss: 2.0949e-09
Epoch 2/1000
33439/33439 [==============================] - 7s 222us/step - loss: 5.1828e-09 - val_loss: 1.7538e-09
Epoch 3/1000
33439/33439 [==============================] - 8s 253us/step - loss: 4.7667e-09 - val_loss: 3.1571e-09
Epoch 4/1000
33439/33439 [==============================] - 9s 278us/step - loss: 4.7305e-09 - val_loss: 1.0058e-08
Epoch 5/1000
33439/33439 [==============================] - 8s 240us/step - loss: 4.6643e-09 - val_loss: 2.8600e-09
Epoch 6/1000
33439/33439 [==============================] - 8s 247us/step - loss: 4.7008e-09 - val_loss: 9.4624e-09
Epoch 7/1000
33439/33439 [==============================] - 7s 209us/step - loss: 4.5151e-09 - val_loss: 2.6374e-09
Epoch 8/1000
33439/33439 [==============================] - 7s 211us/step - loss: 4.5059e-09 - val_loss: 1.3690e-09
Epoch 9/1000
33439/3343

In [17]:
plplot([his.history['loss']])

In [18]:
plplot([his.history['val_loss']])

In [19]:
encoded_ts = encoder.predict(x[50].reshape(1, -1))
decoded_ts = decoder.predict(encoded_ts)

In [20]:
print(encoded_ts)
print(decoded_ts)

[[-0.00166841 -0.00131343  0.00011549 -0.00119899 -0.00047598]]
[[ 1.8197135e-04  3.3186516e-04 -4.3407700e-04 -5.0336431e-04
   2.4100686e-04  1.8419921e-04  2.3837484e-04 -6.0630462e-04
   1.0407002e-03 -1.5523445e-04  1.8901470e-04  6.6676928e-04
  -1.7461646e-04 -3.1761805e-04 -5.5419916e-04 -1.1567085e-03
  -3.6182607e-06  2.0009637e-04 -9.1975951e-04 -1.0249239e-03
  -4.9703702e-04  1.2337409e-04  7.2872313e-04  1.8429457e-05
   9.5620853e-05  6.2338350e-04 -3.6697410e-04  1.9429905e-04
   5.1822595e-04 -3.3671808e-04 -7.0666737e-04 -1.9743004e-04
   8.6942490e-04  6.1302993e-04  1.3136213e-03 -8.0243440e-04
  -3.8046064e-04 -4.9188506e-04 -8.8207715e-05  6.2307436e-06
  -4.8782921e-04  9.7983157e-06 -6.3000026e-04 -3.1482748e-04
   8.1722675e-07  1.7250540e-04 -1.0010318e-03 -1.0495875e-03
   3.3946126e-05 -3.6166297e-04]]


In [21]:
plplot([x[50]])

In [22]:
plplot([decoded_ts[0]])